In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Загрузка данных
classification_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AiDatasets/jobDs.csv")

# Предварительная обработка данных
classification_data = classification_data.dropna()
X_text = classification_data['job_title']
y_class = classification_data['category']

# Преобразование текстовых данных в числовые
vectorizer = TfidfVectorizer(max_features=500)
X_class = vectorizer.fit_transform(X_text).toarray()

# Кодирование целевой переменной
encoder = LabelEncoder()
y_class = encoder.fit_transform(y_class)

# Разделение данных на тренировочные и тестовые
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)

# Обучение модели KNN
knn_class = KNeighborsClassifier(n_neighbors=5)
knn_class.fit(X_train_class, y_train_class)

y_pred_class = knn_class.predict(X_test_class)

# Определение уникальных меток в тестовой выборке
unique_labels = sorted(set(y_test_class))

# Оценка качества
print(classification_report(
    y_test_class,
    y_pred_class,
    labels=unique_labels,  # Указываем метки для тестовой выборки
    target_names=[encoder.inverse_transform([label])[0] for label in unique_labels],
    zero_division=0  # Явно устанавливаем поведение для деления на ноль
))


                                        precision    recall  f1-score   support

                            Accounting       0.50      0.44      0.47         9
       Administration & Office Support       0.69      0.83      0.75       436
             Advertising, Arts & Media       0.67      0.17      0.27        12
          Banking & Financial Services       0.71      0.69      0.70       208
              CEO & General Management       0.80      0.40      0.53        10
        Call Centre & Customer Service       0.54      0.57      0.56        35
                          Construction       0.67      0.76      0.71        85
                 Consulting & Strategy       0.33      0.21      0.26        24
                 Design & Architecture       0.80      0.71      0.75        17
                           Engineering       0.50      0.33      0.40         3
                  Healthcare & Medical       0.00      0.00      0.00         3
         Human Resources & Recruitment 

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Загрузка данных
regression_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AiDatasets/Delhi_v2.csv")

# Удаление ненужных столбцов
X_reg = regression_data.drop(columns=["price", "Address", "desc"])  # Удаляем целевую переменную и текстовые столбцы
y_reg = regression_data["price"]  # Целевая переменная

# Преобразуем категориальные данные в числовые
X_reg = pd.get_dummies(X_reg, drop_first=True)

# Обработка пропущенных значений
imputer = SimpleImputer(strategy="mean")  # Замена NaN на среднее значение
X_reg = imputer.fit_transform(X_reg)

# Нормализация данных
scaler = StandardScaler()
X_reg = scaler.fit_transform(X_reg)

# Разделение данных на тренировочные и тестовые
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Обучение модели KNN
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train_reg, y_train_reg)

# Оценка качества
y_pred_reg = knn_reg.predict(X_test_reg)
print("MAE:", mean_absolute_error(y_test_reg, y_pred_reg))
print("MSE:", mean_squared_error(y_test_reg, y_pred_reg))
print("R^2:", r2_score(y_test_reg, y_pred_reg))


MAE: 1534083.9793281653
MSE: 11355848012919.896
R^2: 0.8084490370416328


In [18]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

# Преобразуем y_train_class в pandas.Series, если это numpy.ndarray
y_train_class = pd.Series(y_train_class)

# Убедимся, что все классы имеют больше одного примера
# Убираем редкие классы с одним экземпляром
class_counts = y_train_class.value_counts()
classes_to_remove = class_counts[class_counts == 1].index
X_train_class_filtered = X_train_class[~y_train_class.isin(classes_to_remove)]
y_train_class_filtered = y_train_class[~y_train_class.isin(classes_to_remove)]

# Создание конвейера с масштабированием и KNN
pipeline_class = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),  # Для совместимости с Tfidf
    ('knn', KNeighborsClassifier())
])

# Подбор гиперпараметров
param_grid_class = {
    'knn__n_neighbors': [3, 5, 7],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

# Использование StratifiedKFold для кросс-валидации с 2 фолдами
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Создание объекта GridSearchCV с указанным StratifiedKFold
grid_class = GridSearchCV(
    pipeline_class,
    param_grid_class,
    cv=cv,
    scoring='accuracy',
    verbose=1,
    return_train_score=True
)

# Подготовка данных (если не сделано ранее)
# X_train_class_filtered, y_train_class_filtered - очищенные от классов с одним экземпляром

print("Начало подбора гиперпараметров...")
grid_class.fit(X_train_class_filtered, y_train_class_filtered)




Начало подбора гиперпараметров...
Fitting 2 folds for each of 12 candidates, totalling 24 fits


GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler',
                                        StandardScaler(with_mean=False)),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': [3, 5, 7],
                         'knn__weights': ['uniform', 'distance']},
             return_train_score=True, scoring='accuracy', verbose=1)

In [5]:
# Вывод промежуточных результатов
print("\nРезультаты кросс-валидации:")
cv_results = grid_class.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Параметры: {params}, Средняя точность: {mean_score:.4f}")

# Лучшие параметры и оценка
print("\nЛучшие параметры для классификации:", grid_class.best_params_)
print("Лучшее качество на кросс-валидации:", grid_class.best_score_)

# Оценка на тестовых данных
print("\nОценка на тестовом наборе:")
best_knn_class = grid_class.best_estimator_
y_pred_class = best_knn_class.predict(X_test_class)
test_accuracy = accuracy_score(y_test_class, y_pred_class)
print(f"Точность на тестовом наборе: {test_accuracy:.4f}")

# Подробный отчет
print("\nКлассификационный отчет:")

# Получаем все классы из y_test_class
unique_classes = np.unique(y_test_class)

# Печатаем отчет, указывая только те классы, которые присутствуют в y_test_class
print(classification_report(
    y_test_class,
    y_pred_class,
    labels=unique_labels,  # Указываем метки для тестовой выборки
    target_names=[encoder.inverse_transform([label])[0] for label in unique_labels],
    zero_division=0  # Явно устанавливаем поведение для деления на ноль
))


Результаты кросс-валидации:
Параметры: {'knn__metric': 'euclidean', 'knn__n_neighbors': 3, 'knn__weights': 'uniform'}, Средняя точность: 0.6675
Параметры: {'knn__metric': 'euclidean', 'knn__n_neighbors': 3, 'knn__weights': 'distance'}, Средняя точность: 0.6807
Параметры: {'knn__metric': 'euclidean', 'knn__n_neighbors': 5, 'knn__weights': 'uniform'}, Средняя точность: 0.6866
Параметры: {'knn__metric': 'euclidean', 'knn__n_neighbors': 5, 'knn__weights': 'distance'}, Средняя точность: 0.6998
Параметры: {'knn__metric': 'euclidean', 'knn__n_neighbors': 7, 'knn__weights': 'uniform'}, Средняя точность: 0.6896
Параметры: {'knn__metric': 'euclidean', 'knn__n_neighbors': 7, 'knn__weights': 'distance'}, Средняя точность: 0.7094
Параметры: {'knn__metric': 'manhattan', 'knn__n_neighbors': 3, 'knn__weights': 'uniform'}, Средняя точность: 0.6883
Параметры: {'knn__metric': 'manhattan', 'knn__n_neighbors': 3, 'knn__weights': 'distance'}, Средняя точность: 0.7023
Параметры: {'knn__metric': 'manhattan',

In [6]:
# Создание конвейера с масштабированием и KNN
pipeline_reg = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor())
])

# Подбор гиперпараметров
param_grid_reg = {
    'knn__n_neighbors': [3, 5, 7],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

grid_reg = GridSearchCV(pipeline_reg, param_grid_reg, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_reg.fit(X_train_reg, y_train_reg)

# Лучшие параметры и оценка
print("Лучшие параметры для регрессии:", grid_reg.best_params_)
print("Лучшее качество на кросс-валидации (MSE):", -grid_reg.best_score_)

# Оценка на тестовых данных
best_knn_reg = grid_reg.best_estimator_
y_pred_reg = best_knn_reg.predict(X_test_reg)
print("MAE:", mean_absolute_error(y_test_reg, y_pred_reg))
print("MSE:", mean_squared_error(y_test_reg, y_pred_reg))
print("R^2:", r2_score(y_test_reg, y_pred_reg))


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Лучшие параметры для регрессии: {'knn__metric': 'euclidean', 'knn__n_neighbors': 5, 'knn__weights': 'uniform'}
Лучшее качество на кросс-валидации (MSE): 8624186163489.498
MAE: 1529077.519379845
MSE: 11304323095607.234
R^2: 0.8093181617002547


In [7]:
class CustomKNNClassifier:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _distance(self, a, b):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((a - b) ** 2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(a - b))
        else:
            raise ValueError("Unsupported metric")

    def predict(self, X):
        predictions = []
        for x in X:
            distances = np.array([self._distance(x, train_x) for train_x in self.X_train])
            nearest_indices = distances.argsort()[:self.n_neighbors]
            nearest_labels = self.y_train[nearest_indices]
            predictions.append(np.bincount(nearest_labels).argmax())
        return np.array(predictions)


In [17]:
custom_knn_class = CustomKNNClassifier(n_neighbors=5)
custom_knn_class.fit(X_train_class, y_train_class)
y_pred_custom_class = custom_knn_class.predict(X_test_class)




In [10]:
unique_classes_in_data = np.unique(np.concatenate([y_test_class, y_pred_custom_class]))

# Подробный отчет с правильными классами
print(classification_report(
    y_test_class,
    y_pred_custom_class,
    labels=unique_classes_in_data,
    target_names=encoder.inverse_transform(unique_classes_in_data),
    zero_division=1  # Это будет предотвращать ошибки деления на 0 и выводить 0 для таких метрик
))

                                        precision    recall  f1-score   support

                            Accounting       0.60      0.33      0.43         9
       Administration & Office Support       0.71      0.82      0.76       436
             Advertising, Arts & Media       0.10      0.17      0.12        12
          Banking & Financial Services       0.67      0.72      0.69       208
              CEO & General Management       0.80      0.40      0.53        10
        Call Centre & Customer Service       0.53      0.51      0.52        35
                          Construction       0.72      0.74      0.73        85
                 Consulting & Strategy       0.38      0.21      0.27        24
                 Design & Architecture       0.80      0.71      0.75        17
                           Engineering       0.50      0.33      0.40         3
                  Healthcare & Medical       1.00      0.00      0.00         3
                 Hospitality & Tourism 

In [11]:
class CustomKNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _distance(self, a, b):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((a - b) ** 2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(a - b))
        else:
            raise ValueError("Unsupported metric")

    def predict(self, X):
        predictions = []
        for x in X:
            distances = np.array([self._distance(x, train_x) for train_x in self.X_train])
            nearest_indices = distances.argsort()[:self.n_neighbors]
            nearest_values = self.y_train[nearest_indices]
            predictions.append(np.mean(nearest_values))
        return np.array(predictions)


In [16]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Преобразуем данные в массивы NumPy без индексов
X_train_reg = np.array(X_train_reg)
y_train_reg = np.array(y_train_reg)
X_test_reg = np.array(X_test_reg)
y_test_reg = np.array(y_test_reg)

# Обучаем модель
custom_knn_reg = CustomKNNRegressor(n_neighbors=5)
custom_knn_reg.fit(X_train_reg, y_train_reg)

# Делаем предсказания
y_pred_custom_reg = custom_knn_reg.predict(X_test_reg)

# Выводим метрики
print("MAE:", mean_absolute_error(y_test_reg, y_pred_custom_reg))
print("MSE:", mean_squared_error(y_test_reg, y_pred_custom_reg))
print("R^2:", r2_score(y_test_reg, y_pred_custom_reg))


MAE: 1534083.9793281653
MSE: 11355848012919.896
R^2: 0.8084490370416328
